In [ ]:
#下载视频
pip3 install you-get
you-get -O ./image/video https://www.bilibili.com/video/BV124411Q7iV

pip3 install moviepy
pip3 install moviepy --upgrade    #TypeError: must be real number, not NoneType
#import moviepy.editor as mpy

In [10]:
%matplotlib inline
from matplotlib import pyplot as plt

import os
import re
import shutil
import cv2
from PIL import Image

In [15]:
#求一个方框图像的平均值
def GridAvage(frame, startX, startY, width, height):
    sum = 0
    for x in range(startX, startX+width):
        for y in range(startY, startY+height):
            sum = sum + frame[y, x]
            frame[y,x]=255   #set to white
    return sum//(width*height)

def GetChar(value):
    symbol = list("$@B%8&WM#*oahkbdpqwmZO0QLCJUYXzcvunxrjft/\|()1{}[]?-_+~<>i!lI;:,\"^`'. ")
    pos = (len(symbol)-1) * value //255
    return symbol[pos]

#把一个像素方框转成字母
def Grid2Char(frame):
    #scale(0.2, 0.5) is very important to display the details
    labelSize=cv2.getTextSize('@', cv2.FONT_HERSHEY_SIMPLEX, 0.3, 1)
    width = labelSize[0][0]
    height = labelSize[0][1] + labelSize[1] 
    
    for x in range(0, frame.shape[1], width):  #Slug!!!
        for y in range(0, frame.shape[0], height):
            w = frame.shape[1] - x
            if w>width: w=width
            
            h = frame.shape[0] - y
            if h>height: h=height
            
            value = GridAvage(frame, x, y, w, h)
            char = GetChar(value)
            value = int (value)
            value = value *3 //5; #darker
            frame = cv2.putText(frame, char, (x, y), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.3, (value, value, value)) #(0, 0, 0)

In [16]:
from IPython.display import clear_output, display

#保存每帧
cap = cv2.VideoCapture("./image/video.mp4")
status,frame = cap.read()

#用于将图片写入视频   #mp4 3 times bigger than flv
videoWriter = cv2.VideoWriter('./image/charDance.mp4', 
                              cv2.VideoWriter_fourcc(*'mp4v'), 
                              cap.get(cv2.CAP_PROP_FPS), 
                              (frame.shape[1],frame.shape[0]), 
                              False)
'''
videoWriter = cv2.VideoWriter('./image/charDance.flv', 
                              cv2.VideoWriter_fourcc(*'FLV1'),  #cap.get(cv2.CAP_PROP_FOURCC),
                              cap.get(cv2.CAP_PROP_FPS),                               
                              (frame.shape[1], frame.shape[0]), #( cap.get(cv2.CAP_PROP_FRAME_WIDTH),  cap.get(cv2.CAP_PROP_FRAME_HEIGHT) ), 
                              False)
'''
index = 0;
while status:
    #if index>=100: break
    clear_output(wait=True)
    print(f"正在处理第{index}帧")    # use display(f) if you encounter performance issues
    #plt.imshow(frame);
    #plt.show()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #plt.imshow(frame, cmap='gray', vmin=0, vmax=255);
    #plt.show()
    #cv2.imwrite('./image/1.png', frame)
    
    Grid2Char(frame)
    #plt.imshow(frame, cmap='gray', vmin=0, vmax=255);
    #plt.show()
    #cv2.imwrite('./image/2.png', frame)
    videoWriter.write(frame)   #新文件大约是原先3倍，two-pass encode?
    status,frame = cap.read()
    index = index + 1

videoWriter.release()
print("Done")


正在处理第99帧
Done


In [ ]:
import moviepy.editor as mpy

#把原先视频变成mp4格式, 好象mpy无法处理flv
#https://video.online-convert.com/convert-to-mp4

def addMusic(musicFileName, myVideoFileName, outputFileName):
    # 读取代码视频
    my_clip = mpy.VideoFileClip(myVideoFileName)
    # 截取背景音乐
    audio_background = mpy.AudioFileClip(musicFileName).subclip(0, 60)
    audio_background.write_audiofile('./image/background.mp3')
    # 视频中插入音频
    final_clip = my_clip.set_audio(audio_background)
    # 保存最终视频
    final_clip.write_videofile(outputFileName, fps=20)
    
addMusic('./image/video.mp4', './image/charDance.mp4', './image/charDanceFinal.mp4')